In [1]:
import numpy as np
import pandas as pd
import re
import itertools
import copy
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
import collections
from collections import defaultdict
import csv

## Расстояние 

In [2]:
def lev_distance(a, b):
    
    # Первыми - строчки 
    # столбики - слово b
    
    global dist_matrix
    
    
    # dis = np.zeros((len(a) + 1, len(b) + 1))
    dis = [[0] * (len(b) + 1) for _ in range(len(a) + 1)] 
    i = 0
    row = 0
    col = 0

    while i < dis.size:

        if row == 0:
            if col != 0:
                dis[row, col] = dis[row, col-1] + 1

        elif col == 0:
            if row != 0:
                dis[row, col] = dis[row - 1, col] + 1
        
  
        elif row > 1 and col > 1 and a[row-1] == b[col-2] and a[row-2] == b[col-1]:
            dis[row, col] = dis[row - 3][col - 3] + 1
            
        
        else:
            dis[row, col] = min([dis[row, col - 1] + 1,  # левый 
                                    dis[row - 1, col - 1] + dist_matrix[a[row-1]][b[col-1]], # диаг               
                                    dis[row - 1, col] + 1]) # верхний
        
        col += 1
        i += 1  
        if col == len(b) + 1:
            col = 0
            row += 1
  
    return dis[len(a), len(b)]

# Данные

In [20]:
data = pd.read_csv('data.csv')
data = data.set_index('Unnamed: 0')

In [26]:
data.head()

,syllabic,stress,long,consonantal,sonorant,continuant,delayed release,approximant,tap,trill,...,anterior,distributed,strident,lateral,DORSAL,high,low,front,back,tense
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
ɒ,+,-,-,-,+,+,None,+,-,-,...,None,None,None,-,+,-,+,-,+,None
a,+,-,-,-,+,+,None,+,-,-,...,None,None,None,-,+,-,+,-,+,None
ɶ,+,-,-,-,+,+,None,+,-,-,...,None,None,None,-,+,-,+,+,-,None
ɑ,+,-,-,-,+,+,None,+,-,-,...,None,None,None,-,+,-,+,-,-,None
œ,+,-,-,-,+,+,None,+,-,-,...,None,None,None,-,+,-,+,+,-,None


# Non Language specific

## data

In [326]:
def sound_dist(a, b, data):
    
    '''
    реализовать stress = + при условии если ударный гласный
    ингнорирование лишней диакритики
    '''
    
    a = pd.get_dummies(data.loc[a]) 
    b = pd.get_dummies(data.loc[b])
    
    together = a + b
    
    similar = 0
    uncommon = 0
    common = 0

    if '+' in together and 2 in together['+'].value_counts():
        similar += together['+'].value_counts()[2]

    if '-' in together and 2 in together['-'].value_counts():
        similar += together['-'].value_counts()[2]

    if 'None' in together:
        None_values = together['None'].value_counts()
        
        if 1 in None_values:
            uncommon = None_values[1]

        if 0 in None_values:
            common = None_values[0]
        
    dist = 1 - (similar / (common + (uncommon * 2)))
    
    return dist


def non_ls_dist_matrix(data):
    
    col_ind = data.index.tolist()
    n_col = len(col_ind)
    dist_matrix = pd.DataFrame(np.zeros((n_col, n_col)),
                               index=col_ind, columns=col_ind)
    
    for i in itertools.combinations_with_replacement(data.index, 2):
            
            result = sound_dist(i[0], i[1], data)
            dist_matrix[i[0]][i[1]] = result
            dist_matrix[i[1]][i[0]] = result
    
    return dist_matrix

In [170]:
%%time
non_ls_dist = non_ls_dist_matrix(141, data)

CPU times: user 43.9 s, sys: 118 ms, total: 44 s
Wall time: 44.1 s


In [171]:
non_ls_dist.head()

,ɒ,a,ɶ,ɑ,œ,Ã,ɔ,o,ɤ,ɘ,...,ɰ,ɰ̠,w,ɥ,j,ɹ,ʋ,ʍ,ɦ,h
ɒ,0.000000,0.086957,0.086957,0.130435,0.173913,0.20,0.12,0.12,0.20,0.24,...,0.407407,0.28,0.20,0.28,0.36,0.575758,0.444444,0.407407,0.586207,0.620690
a,0.086957,0.000000,0.173913,0.043478,0.086957,0.12,0.20,0.20,0.12,0.16,...,0.333333,0.20,0.28,0.36,0.28,0.515152,0.444444,0.481481,0.517241,0.551724
ɶ,0.086957,0.173913,0.000000,0.130435,0.086957,0.28,0.20,0.20,0.28,0.24,...,0.407407,0.36,0.28,0.20,0.28,0.575758,0.444444,0.481481,0.586207,0.620690
ɑ,0.130435,0.043478,0.130435,0.000000,0.043478,0.16,0.24,0.24,0.16,0.12,...,0.333333,0.24,0.32,0.32,0.24,0.515152,0.444444,0.518519,0.517241,0.551724
œ,0.173913,0.086957,0.086957,0.043478,0.000000,0.20,0.28,0.28,0.20,0.16,...,0.333333,0.28,0.36,0.28,0.20,0.515152,0.444444,0.555556,0.517241,0.551724


In [507]:
non_ls_dist.to_csv('/Users/Stoneberry/Desktop/курсач/4/dist_matrix.csv', sep=';')

# Language specific

## data

In [178]:
import csv

In [194]:
def open_data(path, delimiter):
    
    with open(path, 'r') as csvfile:
        
        reader = csv.reader(csvfile, delimiter=delimiter)
        data = list(reader)
    
    return data

In [334]:
words = open_data('test.csv',  ';')

In [482]:
reg = 'ɡ̠ɣ̠|t̠ɬ̠|k̟x̟|g̟ɣ̟|ŋ̟.1|d̠ɮ̠|k̠x̠|d͡ʑ|lò|kp|ɡb|pt|bd|dʒ|dz|dɮ|tʃ|ts|tɬ|ʈʂ|ɖʐ|pf|bv|pɸ|bβ|tθ|dð|cç|ɟʝ|kx|ɡɣ|qχ|ɢʀ|ts.1|tɬ.1|dz.1|dɮ.1|œ.1|ʟ̠|ŋ̟|ʟ̟|ɣ̟|x̟|k̟|ɡ̟|ɣ̠|x̠|k̠|ɡ̠|ɰ̠|ɒ|a|ɶ|ɑ|œ|Ã|ɔ|o|ɤ|ɘ|ə|e|ɞ|ø|ɛ|ɵ|ɯ|u|ʊ|ɨ|ʉ|y|i|ʏ|ɪ|ɴ|ʀ|ɲ|ʎ|ŋ|ʟ|ɳ|ʙ|ɭ|ä|ɻ|ɽ|r|n|m|l|ɾ|ɱ|ʔ|ħ|ʕ|ʁ|q|χ|ɢ|ɕ|ɟ|ʝ|c|ç|tɕ|ɣ|x|k|ɡ|ʑ|ʈ|ɖ|ɬ|ʐ|ɸ|ʂ|ʒ|z|v|t|ʃ|s|p|f|d|b|θ|ɮ|ð|β|ɧ|ɰ|w|ɥ|j|ɹ|ʋ|ʍ|ɦ|h'

# Делим транскрипцию на части

In [490]:
def phone_splitter(word):
    
    '''
        + - advanced
        _ - retracted
        tʲ - мягкий (back +)
        tʷ - Labalized (round + )
        dⁿ - nasal +
        _1 - stress + 
        tˠ - Velarized
        t̬ - voiced
        d̥ - voiceless
    '''
    
    
    def find_stress_vow(word):
        
        '''
        Ищет ударные гласный и дифтонги
        '''
        r_stress = '_[1-4]'
        f = re.findall(r_stress, word)
        st_reg = ''
        
        for i in f:
            st_reg += ('.'*int(i[-1])) + i + '|'
        return st_reg
    
    
    word = re.sub('͡', '_аффрик_', word)
    word = re.sub('̥', '_глух', word)
    word = re.sub('̬', '_звонк', word)
    r_stress = find_stress_vow(word)    # _1

    all_ = '(' + r_stress + '._аффрик_.' + '|' + '._звонк' + '|' + '._глух' + '|' + '.ⁿ' + '|' + '.ʲ' + '|' + '.ʷ' + '|' + reg + '|' + '.'+ ')'
    res = re.findall(all_, word)

    return res + ['<eos>']

In [491]:
word = 'pˠraɒ_2td͡ʑək̟tʟ̠tʷdⁿtʲtˠd̥t̬ʧdʒtʃ'
phone_splitter(word)

['p',
 'ˠ',
 'r',
 'aɒ_2',
 't',
 'd_аффрик_ʑ',
 'ə',
 'k̟',
 't',
 'ʟ̠',
 'tʷ',
 'dⁿ',
 'tʲ',
 't',
 'ˠ',
 'd_глух',
 't_звонк',
 'ʧ',
 'dʒ',
 'tʃ',
 '<eos>']

In [492]:
for i in words:
    for l in i:
        print(phone_splitter(l))

['m', 'e', 'i', 't', '<eos>']
['h', 'e', 'i', 't', '<eos>']
['l', 'a', 'v', '<eos>']
['p', 'r', 'ə', 'k', 't', '<eos>']


In [493]:
def all_phones(arr):

    phones = []

    for line in arr:

        if len(line) != 2:
            raise ValueError('There have to be 2 columns in a line. Check your delimiter.')

        for word in line:

            if word == '':
                raise ValueError('All columns have to be filled.')
            phones += phone_splitter(word)

        phones.append('<\n>')

    return phones

In [494]:
all_phones(words)

['m',
 'e',
 'i',
 't',
 '<eos>',
 'h',
 'e',
 'i',
 't',
 '<eos>',
 '<\n>',
 'l',
 'a',
 'v',
 '<eos>',
 'p',
 'r',
 'ə',
 'k',
 't',
 '<eos>',
 '<\n>']

# irrelevant_features

In [275]:
def irrelevant_features(phones, data):
    
    '''
    Надо ли удалять звуки, которые не встретились?
    '''
    
    features = set()
    
    for index, phone in enumerate(set(phones)):
        
        if phone not in ('<\n>', '<eos>'):
        
            res = set(data.loc[phone].loc[data.loc[phone] == 'None'].keys())
            
            if index == 0: features = res
            else: features &= res

    return features

In [330]:
def ls_dist_matrix(phones, data):
    
    irr_f = irrelevant_features(phones, data)
    data = data.drop(columns=irr_f)
    
    dist_matrix = non_ls_dist_matrix(data)
    
    return dist_matrix

In [331]:
ls_dist = ls_dist_matrix(phones, data)
ls_dist.head()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in long_scalars


,ɒ,a,ɶ,ɑ,œ,Ã,ɔ,o,ɤ,ɘ,...,ɰ,ɰ̠,w,ɥ,j,ɹ,ʋ,ʍ,ɦ,h
ɒ,0.000000,0.105263,0.000000,0.105263,0.105263,0.105263,0.000000,0.000000,0.105263,0.105263,...,0.157895,0.157895,0.052632,0.052632,0.157895,0.44,0.210526,0.333333,0.428571,0.476190
a,0.105263,0.000000,0.105263,0.000000,0.000000,0.000000,0.105263,0.105263,0.000000,0.000000,...,0.052632,0.052632,0.157895,0.157895,0.052632,0.36,0.210526,0.428571,0.333333,0.380952
ɶ,0.000000,0.105263,0.000000,0.105263,0.105263,0.105263,0.000000,0.000000,0.105263,0.105263,...,0.157895,0.157895,0.052632,0.052632,0.157895,0.44,0.210526,0.333333,0.428571,0.476190
ɑ,0.105263,0.000000,0.105263,0.000000,0.000000,0.000000,0.105263,0.105263,0.000000,0.000000,...,0.052632,0.052632,0.157895,0.157895,0.052632,0.36,0.210526,0.428571,0.333333,0.380952
œ,0.105263,0.000000,0.105263,0.000000,0.000000,0.000000,0.105263,0.105263,0.000000,0.000000,...,0.052632,0.052632,0.157895,0.157895,0.052632,0.36,0.210526,0.428571,0.333333,0.380952


# all together

In [2]:
reg = 'ɡ̠ɣ̠|t̠ɬ̠|k̟x̟|g̟ɣ̟|ŋ̟.1|d̠ɮ̠|k̠x̠|d͡ʑ|lò|kp|ɡb|pt|bd|dʒ|dz|dɮ|tʃ|ts|tɬ|ʈʂ|ɖʐ|pf|bv|pɸ|bβ|tθ|dð|cç|ɟʝ|kx|ɡɣ|qχ|ɢʀ|ts.1|tɬ.1|dz.1|dɮ.1|œ.1|ʟ̠|ŋ̟|ʟ̟|ɣ̟|x̟|k̟|ɡ̟|ɣ̠|x̠|k̠|ɡ̠|ɰ̠|ɒ|a|ɶ|ɑ|œ|Ã|ɔ|o|ɤ|ɘ|ə|e|ɞ|ø|ɛ|ɵ|ɯ|u|ʊ|ɨ|ʉ|y|i|ʏ|ɪ|ɴ|ʀ|ɲ|ʎ|ŋ|ʟ|ɳ|ʙ|ɭ|ä|ɻ|ɽ|r|n|m|l|ɾ|ɱ|ʔ|ħ|ʕ|ʁ|q|χ|ɢ|ɕ|ɟ|ʝ|c|ç|tɕ|ɣ|x|k|ɡ|ʑ|ʈ|ɖ|ɬ|ʐ|ɸ|ʂ|ʒ|z|v|t|ʃ|s|p|f|d|b|θ|ɮ|ð|β|ɧ|ɰ|w|ɥ|j|ɹ|ʋ|ʍ|ɦ|h'
diacritics = ['˞', 'ˤ', 'ˠ', 'ʱ', 'ʰ', 'ʴ', 'ʼ', 'ˑ', 'ː', ':', '̤', '̪', '̰', '̺', '̼', '̻', '̚', '̟', '̠', '̈', '̴', '̽', '̝', '̞', '̘', '̙', '̆', '̋', '́', '̄', '̀', '̏'] 
f_changed = ['_аффрик_', '_глух', '_звонк', '_огуб', '_слог', '_неслог']
f_real = ['', '-voice', '+voice', '+round', '+sylabic', '-sylabic']


non_ls_dist = pd.read_csv('/Users/Stoneberry/Desktop/курсач/4/dist_matrix.csv', sep=';')
non_ls_dist = non_ls_dist.set_index('Unnamed: 0')
non_ls_dist.index = non_ls_dist.columns
ftable = pd.read_csv('data.csv')
ftable = ftable.set_index('Unnamed: 0')

In [216]:
vowels = 'ɒ|a|ɶ|ɑ|œ|Ã|ɔ|o|ɤ|ɘ|œ|ə|e|ɞ|ø|ɛ|ɵ|ɯ|u|ʊ|ɨ|ʉ|y|i|ʏ|ɪ)'
vow_d = '̩|̯|̃|'
vow_reg = vowels + '(?:' + vow_d + ')?'


cons = 'ɡ̠ɣ̠|t̠ɬ̠|k̟x̟|g̟ɣ̟|ŋ̟.1|d̠ɮ̠|k̠x̠|d͡ʑ|lò|kp|ɡb|pt|bd|dʒ|dz|dɮ|tʃ|ts|tɬ|ʈʂ|ɖʐ|pf|bv|pɸ|bβ|tθ|dð|cç|ɟʝ|kx|ɡɣ|qχ|ɢʀ|ts.1|tɬ.1|dz.1|dɮ.1|œ.1|ʟ̠|ŋ̟|ʟ̟|ɣ̟|x̟|k̟|ɡ̟|ɣ̠|x̠|k̠|ɡ̠|ɰ̠|ɴ|ʀ|ɲ|ʎ|ŋ|ʟ|ɳ|ʙ|ɭ|ä|ɻ|ɽ|r|n|m|l|ɾ|ɱ|ʔ|ħ|ʕ|ʁ|q|χ|ɢ|ɕ|ɟ|ʝ|c|ç|tɕ|ɣ|x|k|ɡ|ʑ|ʈ|ɖ|ɬ|ʐ|ɸ|ʂ|ʒ|z|v|t|ʃ|s|p|f|d|b|θ|ɮ|ð|β|ɧ|ɰ|w|ɥ|j|ɹ|ʋ|ʍ|ɦ|h)'
cons_reg = cons +'(?:' + '͡'+ '(?:'+ cons + '|͜'+ '(?:' + cons + '|̥|̊|̬|̹|̜|ⁿ|ʲ|ʷ|'+ ')?'

In [48]:
def find_stress_vow(word, phone):
        
    '''
     Ищет ударные гласный и дифтонги
    '''
    r_stress = '_[1-4]'
    f = re.findall(r_stress, word)
    st_reg = ''
        
    for i in f:
        st_reg += ('.'*int(i[-1])) + i + '|'
    return st_reg

In [43]:
find_stress_vow('lɒ_2vtʷ')

'.._2|'

In [36]:
ch = '(?:_аффрик_.|_звонк|_глух|_огуб|ⁿ|ʲ|ʷ|_слог|)?'

In [37]:
d = '(?:˞|ˤ|ˠ|ʱ|ʰ|ʴ|ʼ|ˑ|ː|:|̤|̪|̰|̺|̼|̻|̚|̟|̠|̈|̴|̽|̝|̞|̘|̙|̆|̋|́|̄|̀|̏)?'

In [38]:
re.findall(phone + d + ch, 'lɒ_1vtʷ')

['l', 'ɒ', 'v', 'tʷ']

In [362]:
 class PhoneticAlgorithmIPA:
    
    def __init__(self, data_path, data_delimiter, problem_type):
        
        global non_ls_dist, ftable
        
        self.feature_table = ftable
        self.phones = self.data_splitter(data_path, data_delimiter)
        
        if problem_type == 'LS':
            self.dist_matrix = self.ls_dist_matrix(self.phones, self.feature_table)
       
        if problem_type == 'Non_LS':
            self.dist_matrix = non_ls_dist
        
        else: raise ValueError('Problem type can be either LS or Non_LS.')

        
    def check_phone(self, res):
        
        global diacritics, f_changed, f_real
        
        for index, phone in enumerate(res):
               
            if phone in ('ˌ', 'ˈ', "'"):
                raise ValueError('The stress should be presented this way: V_1')
            
            elif phone in diacritics: continue
                
            elif phone in self.feature_table.index:
                transc.append(phone)
                
            elif 'ʲ' in phone:
                1
            
            elif 'ʷ' in phone:
                1
            
            elif 'ⁿ' in phone:
                1
            
            elif '_' in phone:
                
                try:
                    parts = phone.split('_')
                    
                    f_changed.index()
                
                # ('_аффрик_', '_глух', '_звонк', '_огуб', '_слог', '_неслог')
                except: 1
        
            else:
                raise ValueError('{} consists of irrelevant characters ({}).'.format(''.join(res), phone))

        return transc
    
    
    
    def sound_dist(self, a, b):
            
        a = pd.get_dummies(a) 
        b = pd.get_dummies(b)
            
        together = a + b
        similar = 0
        uncommon = 0
        common = 0

        if '+' in together and 2 in together['+'].value_counts():
            similar += together['+'].value_counts()[2]

        if '-' in together and 2 in together['-'].value_counts():
            similar += together['-'].value_counts()[2]
        
        if 'None' in together:
            None_values = together['None'].value_counts()
        
        if 1 in None_values:
            uncommon = None_values[1]
        
        if 0 in None_values:
            common = None_values[0]
        
        dist = 1 - (similar / (common + (uncommon * 2)))
        return dist

    
    def vowel_trans(self, a):
        
        if '̃' in a:  # nasal +
            a = copy.copy(self.feature_table.loc[a])
            a['nasal'] = '+'
        
        if '̩' in a:  # sylabic
            a = copy.copy(self.feature_table.loc[a])
            a['syllabic'] = '+'
        
        if '̯' in a:  # non sylabic
            a = copy.copy(self.feature_table.loc[a])
            a['syllabic'] = '-'
        
        return a
         
    
    
    def check_phone(self, a):
        
        real = re.findall(reg, a)
        
        if '͡' in a or '͜' in a:  # aff
            if a != 'd͡ʑ':
                a = [copy.copy(self.feature_table.loc[i]) for i in real]
            return a
    
        if '_' in a:
            a = [self.vowel_trans(i) for i in real]

        if 'ʲ' in a:  # мягкий (back +)
            a = copy.copy(self.feature_table.loc[real[0]])
            a['back'] = '+'
            
        if 'ʷ' in a:  # Labalized (round + )
            a = copy.copy(self.feature_table.loc[real[0]])
            a['round'] = '+'
            
        if 'ⁿ' in a or '̃' in a:  # nasal +
            a = copy.copy(self.feature_table.loc[real[0]])
            a['nasal'] = '+'
            
        if '̥' in a or '̊' in a:  # voiceless
            a = copy.copy(self.feature_table.loc[real[0]])
            a['voice'] = '-'
            
        if '̬' in a:  # voiced
            a = copy.copy(self.feature_table.loc[real[0]])
            a['voice'] = '+'
            
        if '̹' in a or '̜' in a:  # rounded 
            a = copy.copy(self.feature_table.loc[real[0]])
            a['round'] = '+'
        
        a = self.vowel_trans(a)
        
        return a
    
    
    def comparison(self, a, b):
        
        if type(a) == str and type(b) == str:
            res = self.dist_matrix[a][b]
                
        elif type(a) != str and type(b) != str:
            res = self.sound_dist(self.feature_table.loc[a], self.feature_table.loc[b])
            
        elif type(a) != str:
            res = self.sound_dist(a, self.feature_table.loc[b])
    
        elif type(b) != str:
            res = self.sound_dist(self.feature_table.loc[a], b)
        
        return res
        
        
    def dift_affr(self, a, b):
        
        res = []
        
        if len(a) == len(b):  # ai vs ae
            for index, item in enumerate(a):
                res.append(self.comparison(item, b[index]))
            return sum(res)
        
        
        for item in a:  # aie vs ae
            for itm in b:
                res.append(self.comparison(item, itm))
   
        return min(res) + abs(len(a) - len(b))
    
    
    def phone_dist(self, a, b):
        
        print(a, b)
        
        a = self.check_phone(a)
        b = self.check_phone(b)
        
        if type(a) == list and type(b) == list:
            res = self.dift_affr(a, b)
          
        elif type(a) == list:
            res = self.dift_affr(a, [b])
            
        elif type(b) == list:
            res = self.dift_affr([a], b)
        
        else:
            res = self.comparison(a, b)
        
        return res
        
        
    def phone_splitter(self, word):
    
        '''
            + - advanced
            _ - retracted
            tʲ - мягкий (back +)
            tʷ - Labalized (round + )
            dⁿ - nasal +
            _1 - stress + 
            tˠ - Velarized
            t̬ - voiced
            d̥ - voiceless
            ŋ̊ - voiceless
            ɔ̹ - rounded 
            ɔ̜ - rounded 
            m̩ - sylabic
            e̯ - non sylabic
            x͜x 
        '''
    
        global vow_reg, cons_reg
    
        def find_stress_vow(word, phone):
        
            '''
            Ищет ударные гласный и дифтонги
            '''
            r_stress = '_[1-4]'
            f = re.findall(r_stress, word)
            st_reg = ''
        
            for i in f:
                st_reg += (phone*int(i[-1])) + i 
            return st_reg
   
        word = re.sub('˞|ˤ|ˠ|ʱ|ʰ|ʴ|ʼ|ˑ|ː|:|̤|̪|̰|̺|̼|̻|̚|̟|̠|̈|̴|̽|̝|̞|̘|̙|̆|̋|́|̄|̀|̏', '', word)
        r_stress = find_stress_vow(word, vow_reg)    # ._1

        irrelevant = '(?!' + reg + '|' + vow_d + '͡|͜|̥|̊|̬|̹|̜|ⁿ|ʲ|ʷ|(?:_|[1-9])' + ').'
        res = re.findall(irrelevant , word)
        if res != []:
            raise ValueError('"{}" contains irrelevant symbols: {}'.format(word, ''.join(res)))

        if r_stress == '':  # вызвать ошибку???
            all_ = '(?:' + vow_reg  + '|(?:' + cons_reg
        else:
            all_ = '(?:' + r_stress  + '|(?:' + vow_reg + '|(?:' + cons_reg
            
        return re.findall(all_ , word)
        
        
        
    def all_phones(self, arr):

        phones = []

        for line in arr:
            
            ph_line = []
            
            if len(line) != 2:
                raise ValueError('There have to be 2 columns in a line. Check your delimiter.')

            for word in line:
                if word == '':
                     raise ValueError('All columns have to be filled.')
                ph_line.append(self.phone_splitter(word))
            phones.append(ph_line)

        return phones
    
        
        
    def data_splitter(self, data_path, data_delimiter):
            
        with open(data_path, 'r') as csvfile:
            reader = csv.reader(csvfile, delimiter=data_delimiter)
            data = list(reader)
 
        phones = self.all_phones(data)
            
        return phones
        
        
    def rebuild_dist_matrix(self):
        return
        
        
        
    
    def lev_distance(self, a, b):
    
        # Первыми - строчки 
        # столбики - слово b
    
        dis = np.zeros((len(a) + 1, len(b) + 1))
        i = 0
        row = 0
        col = 0

        while i < dis.size:

            if row == 0:
                if col != 0:
                    dis[row, col] = dis[row, col-1] + 1

            elif col == 0:
                if row != 0:
                    dis[row, col] = dis[row - 1, col] + 1
        
            elif row > 1 and col > 1 and a[row-1] == b[col-2] and a[row-2] == b[col-1]:
                dis[row, col] = dis[row - 3][col - 3] + 1
        
            else: #  self.dist_matrix[a[row-1]][b[col-1]]
                dis[row, col] = np.min([dis[row, col - 1] + 1,  # левый 
                                    dis[row - 1, col - 1] + self.phone_dist(a[row-1], b[col-1]), # диаг               
                                    dis[row - 1, col] + 1]) # верхний
        
            col += 1
            i += 1  
            if col == len(b) + 1:
                col = 0
                row += 1
  
        return dis[len(a), len(b)]
        
        
        
        

        
        
        

    def non_ls_dist_matrix(self, data):
            
        col_ind = data.index.tolist()
        n_col = len(col_ind)
        dist_matrix = pd.DataFrame(np.zeros((n_col, n_col)),
                               index=col_ind, columns=col_ind)
        for i in itertools.combinations_with_replacement(data.index, 2):
            result = self.sound_dist(i[0], i[1], data)
            dist_matrix[i[0]][i[1]] = result
            dist_matrix[i[1]][i[0]] = result
        return dist_matrix
        
        
            
            
            
    def irrelevant_features(self, phones, data):
        '''
        Надо ли удалять звуки, которые не встретились?
        
        ПЕРЕДЕЛАТЬ В ЗАВИСИМОСТИ ОТ ТОГО, КАК ВЫГЛЯДЯТ PHONES
        
        '''
        features = set()
        for index, phone in enumerate(set(phones)):
               
            if phone not in ('<\n>', '<eos>'):
                res = set(data.loc[phone].loc[data.loc[phone] == 'None'].keys())
                if index == 0: features = res
                else: features &= res
                    
        return features
        
        
        
        
    def ls_dist_matrix(self, phones, data):
        irr_f = self.irrelevant_features(phones, data)
        data = data.drop(columns=irr_f)
        dist_matrix = non_ls_dist_matrix(data)
        return dist_matrix 
    
    
    
    def phonetic_distance(self):
        
        res = []
        
        for pair in self.phones:
            print(pair[0], pair[1])
            score = self.lev_distance(pair[0], pair[1])
            print(score)
            res.append(score)
        
        return res
        

In [363]:
ipa = PhoneticAlgorithmIPA('/Users/Stoneberry/Desktop/курсач/4/test.csv', ';', 'Non_LS')


ipa.phonetic_distance()

['m', 'ẽ_1', 'i', 't'] ['h', 'e', 'i', 't', 'pʷ']
m h
m e
m i
m t
m pʷ
ẽ_1 h
ẽ_1 e
ẽ_1 i
ẽ_1 t
ẽ_1 pʷ
i h
i e
i i
i t
i pʷ
t h
t e
t i
t t
t pʷ
1.4285714285714286
['l', 'ɒ_1', 'v', 'tʷ'] ['p', 'r', 'd͡ʑ', 'ə', 'k', 't']
l p
l r
l d͡ʑ
l ə
l k
l t
ɒ_1 p
ɒ_1 r
ɒ_1 d͡ʑ
ɒ_1 ə
ɒ_1 k
ɒ_1 t
v p
v r
v d͡ʑ
v ə
v k
v t
tʷ p
tʷ r
tʷ d͡ʑ
tʷ ə
tʷ k
tʷ t
2.7910540184453225


[1.4285714285714286, 2.7910540184453225]

In [357]:
for i in [ipa.feature_table.loc['a']]:
    print(i)
    print('====')

syllabic              +
stress                -
long                  -
consonantal           -
sonorant              +
continuant            +
delayed release    None
approximant           +
tap                   -
trill                 -
nasal                 -
voice                 +
spread gl             -
constr gl             -
LABIAL                -
round                 -
labiodental           -
CORONAL               -
anterior           None
distributed        None
strident           None
lateral               -
DORSAL                +
high                  -
low                   +
front                 -
back                  +
tense              None
Name: a, dtype: object
====


In [116]:
list('a̤d̪a̰d̺d̼d̻t̚u̟e̠ël̴ɫe̽e̝β̞e̘e̙ĕe̋éēèȅ')

['a',
 '̤',
 'd',
 '̪',
 'a',
 '̰',
 'd',
 '̺',
 'd',
 '̼',
 'd',
 '̻',
 't',
 '̚',
 'u',
 '̟',
 'e',
 '̠',
 'e',
 '̈',
 'l',
 '̴',
 'ɫ',
 'e',
 '̽',
 'e',
 '̝',
 'β',
 '̞',
 'e',
 '̘',
 'e',
 '̙',
 'e',
 '̆',
 'e',
 '̋',
 'e',
 '́',
 'e',
 '̄',
 'e',
 '̀',
 'e',
 '̏']

In [ ]:
d̪a̰d̺d̼d̻t̚u̟e̠ël̴e̽e̝β̞e̘e̙ĕe̋éēèȅa̤

In [118]:
ẽ'['̤',  '̪',  '̰',  '̺',  '̼',  '̻',  '̚',  '̟',  '̠',  '̈',  '̴',   '̽',  '̝',  '̞',  '̘',  '̙',  '̆',  '̋',  '́',  '̄',  '̀',  '̏'])

'̴̤̪̰̺̼̻̟̠̝̞̘̙̈̽̆̋́̄̀̏̚'

In [124]:
list('˞ˤˠʱʰʴʼˑː:').index

ValueError: 'a' is not in list

In [349]:
re.findall(reg, 'e̯a_2')

['e', 'a']

In [44]:
list('ẽ')

['e', '̃']

In [ ]:
word = re.sub('͡|͜', '_аффрик_', word)
word = re.sub('̥|̊', '_глух', word)
word = re.sub('̬', '_звонк', word)
word = re.sub('̹|̜', '_огуб', word)
word = re.sub('̩', '_слог', word)
word = re.sub('̯', '_неслог', word)
word = re.sub('̃', '_нос', word)

In [47]:
def find_stress_vow(word, phone):
        
            '''
            Ищет ударные гласный и дифтонги
            '''
            r_stress = '_[1-4]'
            f = re.findall(r_stress, word)
            st_reg = ''
        
            for i in f:
                st_reg += (phone*int(i[-1])) + i + '|'
            return st_reg
 


'̹|̜'

In [51]:
phone = '(?:' + reg + ')'
v = find_stress_vow('ẽ_1', phone)
re.findall(v, 'ẽ_1')

['', '̃_1', '']

In [60]:
reg

'ɡ̠ɣ̠|t̠ɬ̠|k̟x̟|g̟ɣ̟|ŋ̟.1|d̠ɮ̠|k̠x̠|d͡ʑ|lò|kp|ɡb|pt|bd|dʒ|dz|dɮ|tʃ|ts|tɬ|ʈʂ|ɖʐ|pf|bv|pɸ|bβ|tθ|dð|cç|ɟʝ|kx|ɡɣ|qχ|ɢʀ|ts.1|tɬ.1|dz.1|dɮ.1|œ.1|ʟ̠|ŋ̟|ʟ̟|ɣ̟|x̟|k̟|ɡ̟|ɣ̠|x̠|k̠|ɡ̠|ɰ̠|ɒ|a|ɶ|ɑ|œ|Ã|ɔ|o|ɤ|ɘ|ə|e|ɞ|ø|ɛ|ɵ|ɯ|u|ʊ|ɨ|ʉ|y|i|ʏ|ɪ|ɴ|ʀ|ɲ|ʎ|ŋ|ʟ|ɳ|ʙ|ɭ|ä|ɻ|ɽ|r|n|m|l|ɾ|ɱ|ʔ|ħ|ʕ|ʁ|q|χ|ɢ|ɕ|ɟ|ʝ|c|ç|tɕ|ɣ|x|k|ɡ|ʑ|ʈ|ɖ|ɬ|ʐ|ɸ|ʂ|ʒ|z|v|t|ʃ|s|p|f|d|b|θ|ɮ|ð|β|ɧ|ɰ|w|ɥ|j|ɹ|ʋ|ʍ|ɦ|h'

In [77]:
def find_stress_vow(word, phone):
        
    '''
     Ищет ударные гласный и дифтонги
    '''
    r_stress = '_[1-4]'
    f = re.findall(r_stress, word)
    st_reg = ''
        
    for i in f:
        st_reg += (phone*int(i[-1])) + i 
    return st_reg

In [107]:
vowels = '(?:ɒ|a|ɶ|ɑ|œ|Ã|ɔ|o|ɤ|ɘ|œ|ə|e|ɞ|ø|ɛ|ɵ|ɯ|u|ʊ|ɨ|ʉ|y|i|ʏ|ɪ)'
vow_reg = vowels + '(?:' + '̩|̯|̃' + ')?'


v = find_stress_vow('ẽ_1', vow_reg)
re.findall(v, 'ẽa_1')

['a_1']

In [225]:
vowels = 'ɒ|a|ɶ|ɑ|œ|Ã|ɔ|o|ɤ|ɘ|œ|ə|e|ɞ|ø|ɛ|ɵ|ɯ|u|ʊ|ɨ|ʉ|y|i|ʏ|ɪ)'
vow_d = '̩|̯|̃|'
vow_reg = vowels + '(?:' + vow_d + ')?'


cons = 'ɡ̠ɣ̠|t̠ɬ̠|k̟x̟|g̟ɣ̟|ŋ̟.1|d̠ɮ̠|k̠x̠|d͡ʑ|lò|kp|ɡb|pt|bd|dʒ|dz|dɮ|tʃ|ts|tɬ|ʈʂ|ɖʐ|pf|bv|pɸ|bβ|tθ|dð|cç|ɟʝ|kx|ɡɣ|qχ|ɢʀ|ts.1|tɬ.1|dz.1|dɮ.1|œ.1|ʟ̠|ŋ̟|ʟ̟|ɣ̟|x̟|k̟|ɡ̟|ɣ̠|x̠|k̠|ɡ̠|ɰ̠|ɴ|ʀ|ɲ|ʎ|ŋ|ʟ|ɳ|ʙ|ɭ|ä|ɻ|ɽ|r|n|m|l|ɾ|ɱ|ʔ|ħ|ʕ|ʁ|q|χ|ɢ|ɕ|ɟ|ʝ|c|ç|tɕ|ɣ|x|k|ɡ|ʑ|ʈ|ɖ|ɬ|ʐ|ɸ|ʂ|ʒ|z|v|t|ʃ|s|p|f|d|b|θ|ɮ|ð|β|ɧ|ɰ|w|ɥ|j|ɹ|ʋ|ʍ|ɦ|h)'
cons_reg = cons +'(?:' + '͡'+ '(?:'+ cons + '|͜'+ '(?:' + cons + '|̥|̊|̬|̹|̜|ⁿ|ʲ|ʷ|'+ ')?'


word = 'lɒ_1vtʷ'
v = find_stress_vow(word, vow_reg)

if v == '':
    all_ = '(?:' + vow_reg  + '|(?:' + cons_reg
else:
    all_ = '(?:' + v  + '|(?:' + vow_reg + '|(?:' + cons_reg

    
re.findall( all_ , word)

['l', 'ɒ_1', 'v', 'tʷ']

In [227]:
v

'ɒ|a|ɶ|ɑ|œ|Ã|ɔ|o|ɤ|ɘ|œ|ə|e|ɞ|ø|ɛ|ɵ|ɯ|u|ʊ|ɨ|ʉ|y|i|ʏ|ɪ)(?:̩|̯|̃|)?_1'

In [277]:
word = "lɒ_1vtʷы'"


 
all_ = '(?!' + reg + '|' + vow_d + '͡|͜|̥|̊|̬|̹|̜|ⁿ|ʲ|ʷ|(?:_|[1-9])' + ').'

re.findall(all_ , word)

['ы', "'"]

In [232]:
all_

'(?!_.[1-9]|ɡ̠ɣ̠|t̠ɬ̠|k̟x̟|g̟ɣ̟|ŋ̟.1|d̠ɮ̠|k̠x̠|d͡ʑ|lò|kp|ɡb|pt|bd|dʒ|dz|dɮ|tʃ|ts|tɬ|ʈʂ|ɖʐ|pf|bv|pɸ|bβ|tθ|dð|cç|ɟʝ|kx|ɡɣ|qχ|ɢʀ|ts.1|tɬ.1|dz.1|dɮ.1|œ.1|ʟ̠|ŋ̟|ʟ̟|ɣ̟|x̟|k̟|ɡ̟|ɣ̠|x̠|k̠|ɡ̠|ɰ̠|ɒ|a|ɶ|ɑ|œ|Ã|ɔ|o|ɤ|ɘ|ə|e|ɞ|ø|ɛ|ɵ|ɯ|u|ʊ|ɨ|ʉ|y|i|ʏ|ɪ|ɴ|ʀ|ɲ|ʎ|ŋ|ʟ|ɳ|ʙ|ɭ|ä|ɻ|ɽ|r|n|m|l|ɾ|ɱ|ʔ|ħ|ʕ|ʁ|q|χ|ɢ|ɕ|ɟ|ʝ|c|ç|tɕ|ɣ|x|k|ɡ|ʑ|ʈ|ɖ|ɬ|ʐ|ɸ|ʂ|ʒ|z|v|t|ʃ|s|p|f|d|b|θ|ɮ|ð|β|ɧ|ɰ|w|ɥ|j|ɹ|ʋ|ʍ|ɦ|h|̩|̯|̃|͡|͜|̥|̊|̬|̹|̜|ⁿ|ʲ|ʷ).'

In [204]:
reg

'ɡ̠ɣ̠|t̠ɬ̠|k̟x̟|g̟ɣ̟|ŋ̟.1|d̠ɮ̠|k̠x̠|d͡ʑ|lò|kp|ɡb|pt|bd|dʒ|dz|dɮ|tʃ|ts|tɬ|ʈʂ|ɖʐ|pf|bv|pɸ|bβ|tθ|dð|cç|ɟʝ|kx|ɡɣ|qχ|ɢʀ|ts.1|tɬ.1|dz.1|dɮ.1|œ.1|ʟ̠|ŋ̟|ʟ̟|ɣ̟|x̟|k̟|ɡ̟|ɣ̠|x̠|k̠|ɡ̠|ɰ̠|ɒ|a|ɶ|ɑ|œ|Ã|ɔ|o|ɤ|ɘ|ə|e|ɞ|ø|ɛ|ɵ|ɯ|u|ʊ|ɨ|ʉ|y|i|ʏ|ɪ|ɴ|ʀ|ɲ|ʎ|ŋ|ʟ|ɳ|ʙ|ɭ|ä|ɻ|ɽ|r|n|m|l|ɾ|ɱ|ʔ|ħ|ʕ|ʁ|q|χ|ɢ|ɕ|ɟ|ʝ|c|ç|tɕ|ɣ|x|k|ɡ|ʑ|ʈ|ɖ|ɬ|ʐ|ɸ|ʂ|ʒ|z|v|t|ʃ|s|p|f|d|b|θ|ɮ|ð|β|ɧ|ɰ|w|ɥ|j|ɹ|ʋ|ʍ|ɦ|h'

In [ ]:
'(?!'

In [179]:
re.findall('(?!ɒ|a|ɶ|ɑ|œ|Ã|ɔ|o|ɤ|ɘ|œ|ə|e|ɞ|ø|ɛ|ɵ|ɯ|u|ʊ|ɨ|ʉ|y|i|ʏ|ɪ)(?:̩|̯|̃)?.', 'asasaaaaд')

['s', 's', 'д']

In [180]:
v

'ɒ|a|ɶ|ɑ|œ|Ã|ɔ|o|ɤ|ɘ|œ|ə|e|ɞ|ø|ɛ|ɵ|ɯ|u|ʊ|ɨ|ʉ|y|i|ʏ|ɪ)(?:̩|̯|̃)?_1'

In [71]:
re.findall(r, 'ẽe̯')

['ẽ', 'e̯']

In [76]:
r

'(?:ɒ|a|ɶ|ɑ|œ|Ã|ɔ|o|ɤ|ɘ|œ|ə|e|ɞ|ø|ɛ|ɵ|ɯ|u|ʊ|ɨ|ʉ|y|i|ʏ|ɪ)(?:̩|̯|̃)?'

In [279]:
a = ['͡|͜', '̥|̊', '̬', '̹|̜', 'ⁿ|ʲ|ʷ|', '̩|̯|̃|']

In [280]:
for i in a:
    print(list(i))

['͡', '|', '͜']
['̥', '|', '̊']
['̬']
['̹', '|', '̜']
['ⁿ', '|', 'ʲ', '|', 'ʷ', '|']
['̩', '|', '̯', '|', '̃', '|']


In [ ]:
t̬ - voiced
            d̥ - voiceless
            ŋ̊ - voiceless
            ɔ̹ - rounded 
            ɔ̜ - rounded 
            m̩ - sylabic
            e̯ - non sylabic
            x͜x 